컴퓨터는 숫자로 바꿔줘야 잘처리한다. 따라서 텍스트를 숫자로 바꾸는 여러가지 기법들이 존재했다. 본격적으로 배우기 앞서서 차근차근 배워보자
첫번째로는 각 단어를 고유한 정수에 매핑시켜야할 때가 있다. 단어가 5000개라면 1번부터 5000번까지 말이다.
인덱스를 부여하는 방법은 랜덤하게 할 수 있지만 보통 단어의 등장 빈도수로 정렬하고 부여하는 방법을 주로 사용한다.

1. 정수인코딩 (Integet Encoding)
단어를 빈도순으로 정렬하고 높은 순서부터 부여하는 방법이다.

In [1]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

sentence_token = sent_tokenize(raw_text)
print(sentence_token)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [2]:
vocab = {}
stop_words = set(stopwords.words('english'))
prepro_data = []

for sent in sentence_token:
    result = []
    word_token = word_tokenize(sent)
    for word in word_token:
        word = word.lower()
        if word not in stop_words:
            if len(word) >2:
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                
                vocab[word] += 1
    prepro_data.append(result)

print('불용어제거, 정규화, 워드토큰 등 전처리 이후 데이터 : ',prepro_data)
print('카운팅된 단어 : ',vocab)

불용어제거, 정규화, 워드토큰 등 전처리 이후 데이터 :  [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
카운팅된 단어 :  {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


높은 단어면 0 낮으면 그 다음으로 인코딩해보자

In [3]:
encoding = {}
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)

i = 0
for (word, freq) in vocab_sorted:
    i +=1
    encoding[word] = i

print(encoding)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


이 과정을 편하게 해줄 nltk의 FreqDist를 사용해보자 

In [4]:
from nltk import FreqDist
import numpy as np

#hstack으로 전처리한 데이터 문장 구분을 제거해주기

vocab = FreqDist(np.hstack(prepro_data))

print(vocab)

print(vocab['barber']) #빈도수 출력

<FreqDist with 13 samples and 36 outcomes>
8


In [5]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) #등장 빈도수가 높은 상위 5개 단어만 저장하기

print(vocab)


[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [6]:
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


keras를 사용해서 텍스트 전처리를 해보자 
정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저를 사용하기도 하는데 사용방법과 그 특징을 알아보자

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(prepro_data) #이 코드 실행 시 빈도수 기준으로 단어 집합 생성
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


단어 빈도수가 높은대로 인덱스가 부여된 것을 확인할 수 있다. 카운팅이 어떻게 되었는지 확인하고 싶다면
word_counts를 사용하면 된다.

In [9]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [11]:
print(tokenizer.texts_to_sequences(prepro_data))


[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


위와 같이 전처리 된 데이터를 texts_to_sequences에 통과시키면 정수 인코딩 된걸로 들어가 있는 것을 확인할 수 있다 
가장 높은것 n개로 사용하려면 Tokenizer(num_words = 숫자) 와 같이 사용하면 된다.

In [14]:
tokenizer = Tokenizer(num_words=5+1)
tokenizer.fit_on_texts(prepro_data)
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(prepro_data))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


위와 같이 하면 1-5번째만 사용해서 인코딩된 것을 확인할 수 있다. 또한 Out Of Vocab 인 OOV를 포함시켜서 인코딩하고 싶다면 다음과 같이 진행할 수 있다.

In [15]:
tokenizer = Tokenizer(num_words=5+2,oov_token='OOV') #1부터 시작하게 +1해줬고 OOV 자리 +1 해줬다.
tokenizer.fit_on_texts(prepro_data)
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(prepro_data))

{'OOV': 1, 'barber': 2, 'secret': 3, 'huge': 4, 'kept': 5, 'person': 6, 'word': 7, 'keeping': 8, 'good': 9, 'knew': 10, 'driving': 11, 'crazy': 12, 'went': 13, 'mountain': 14}
OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])
[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
